In [5]:
%load_ext autoreload
%autoreload 2
# import os, sys
# import glob
# import numpy as np
# import pandas as pd
# import torch

sys.path.append('/dartfs/rc/lab/F/FinnLab/tommy/isc_asynchrony_behavior/code/utils/')
sys.path.append('/dartfs/rc/lab/F/FinnLab/tommy/isc_asynchrony_behavior/code/modeling/joint-clm-prosody/')

from config import *
# from src.data.components.audio_text_dataset import AudioTextDataset, load_audio, parse_textgrid, process_wavelet_file, extract_word_segment, pool_embeddings

# from torch.utils.data import DataLoader
# from src.data.components.collators import audio_text_collator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Set up dataset and create dataloader

In [ ]:
# DATASET = 'pfka-moth-stories'
# split = 'train'
DATASET = 'gigaspeech/m'
split = 'test'
text_model_name = 'gpt2'
audio_model_name = 'wav2vec2'

dataset_dir = os.path.join(DATASETS_DIR, 'nlp-datasets', DATASET)
cache_dir = os.path.join(SCRATCH_DIR, 'nlp-datasets', DATASET)

# create datasets
dataset = AudioTextDataset(
    dataset_dir=dataset_dir,
    cache_dir=cache_dir,
    audio_model_name=audio_model_name, 
    text_model_name=text_model_name, 
    split=split,
)

dataset.preprocess_data()

dataset._initialize_models()

Metadata exists. Processing only new files...
No new files to process


### Create segments for the current item

In [ ]:
 # Load and validate all file data
fn = dataset.file_names[0]
file_data = dataset._load_file_data(fn)

#### Get text tokens

In [17]:
# Process text through tokenizer and get token information
text = " ".join([word['text'] for word in file_data['words']])
text_tokens = dataset.text_tokenizer(text)

# Get token counts and associated word ids
word_ids, token_counts = np.unique(text_tokens.word_ids(), return_counts=True)

#### Segment audio word level segments

In [24]:
words = file_data['words']
waveform = file_data['waveform']
sample_rate = 16000

word_ids, token_counts = np.unique(text_tokens.word_ids(), return_counts=True)

segments = []

for word, idx, n_tokens in zip(words, word_ids, token_counts):
    if n_tokens > 1:
        ratios = torch.tensor([len(x) for x in self.text_tokenizer.batch_decode(
            text_tokens['input_ids'][idx:idx+n_tokens])])
        ratios = ratios / ratios.sum()
        word_segments = extract_word_segment(waveform, sample_rate, 
                                            word["start"], word["end"], ratios=ratios)
    else:
        word_segments = extract_word_segment(waveform, sample_rate, 
                                            word["start"], word["end"])
    segments.extend(word_segments)

### Check padded inputs

#### Load models

#### wav2vec2 comparison 

The problem can be [found here](https://github.com/huggingface/transformers/issues/21534) in a github issue

In [84]:
from transformers import AutoProcessor, AutoModel

## This version exhibits differences based on padding
# audio_model_name = "facebook/wav2vec2-base-960h"

# This version does not exhibit differences
audio_model_name = "facebook/wav2vec2-large-960h-lv60"


processor = AutoProcessor.from_pretrained(audio_model_name)
audio_model = AutoModel.from_pretrained(audio_model_name)

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60 and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### data2vec comparison

In [ ]:
from transformers import AutoProcessor, AutoModel

## This version exhibits differences based on padding
# audio_model_name = "facebook/data2vec-audio-base-960h"

## Both models exhibit differences
audio_model_name = "patrickvonplaten/data2vec-base-960h" #"facebook/data2vec-audio-large-100h"

processor = AutoProcessor.from_pretrained(audio_model_name)
audio_model = AutoModel.from_pretrained(audio_model_name)

preprocessor_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/373M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/373M [00:00<?, ?B/s]

#### Run with padded

In [87]:
# Pad the batch
padded_features = processor(segments, sampling_rate=sample_rate, padding=True, return_attention_mask=True, return_tensors="pt")

with torch.no_grad():
    padded_outs = audio_model(**padded_features).last_hidden_state

attention_mask = audio_model._get_feature_vector_attention_mask(
    padded_outs.shape[1], 
    padded_features['attention_mask']
)

padded_embeds = pool_embeddings(padded_outs, attention_mask)

#### Run without padding

In [88]:
# Pad the batch
no_pad_features = processor(segments[:1], sampling_rate=sample_rate,return_attention_mask=True, return_tensors="pt")

with torch.no_grad():
    no_pad_outs = audio_model(**no_pad_features).last_hidden_state

attention_mask = audio_model._get_feature_vector_attention_mask(
    no_pad_outs.shape[1], 
    no_pad_features['attention_mask']
)

no_pad_embed = pool_embeddings(no_pad_outs, attention_mask)

#### Compare embeddings

In [89]:
torch.cosine_similarity(padded_embeds[:1], no_pad_embed[:1])

tensor([0.9360])

## Test video chopping

In [ ]:
%load_ext autoreload
%autoreload 2
import os, sys
import glob
import numpy as np
import pandas as pd
import torch

sys.path.append('/dartfs/rc/lab/F/FinnLab/tommy/isc_asynchrony_behavior/code/utils/')
sys.path.append('/dartfs/rc/lab/F/FinnLab/tommy/isc_asynchrony_behavior/code/modeling/preproc-datasets/')

from config import *
import utils

### Create dataset

In [5]:
%load_ext autoreload
%autoreload 2

import os, sys
import glob
import argparse
import math
import json
import numpy as np
import torch
from tqdm import tqdm
from torchvision.io import read_video
import shutil
# from transformers import AutoImageProcessor, AutoProcessor, AutoModel, AutoTokenizer
sys.path.append('/dartfs/rc/lab/F/FinnLab/tommy/isc_asynchrony_behavior/code/modeling/preproc-datasets/')
sys.path.append('/dartfs/rc/lab/F/FinnLab/tommy/isc_asynchrony_behavior/code/modeling/preproc-datasets/feature-extraction/')

import utils

from extract_dataset_features import (
    initialize_models,
    load_file_data,
    process_video,
    process_single_file
)

sys.path.append('/dartfs/rc/lab/F/FinnLab/tommy/isc_asynchrony_behavior/code/utils/')
# sys.path.append('/dartfs/rc/lab/F/FinnLab/tommy/isc_asynchrony_behavior/code/modeling/joint-clm-prosody/')

from config import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [110]:
from types import SimpleNamespace

def get_args(dataset):
    args = SimpleNamespace(
        text_model = 'gpt2',
        audio_model = 'wav2vec2',
        video_model = 'data2vec',
        split='train',
        num_shards=5,
        current_shard=0,
        min_words=4,
        max_words=128,
        overwrite=0,
        output_dir=os.path.join(DATASETS_DIR, 'nlp-datasets', dataset)
    )
    return args

# Using the args
dataset = 'lrs3'
args = get_args(dataset)


In [112]:
dataset_dir = os.path.join(DATASETS_DIR, 'nlp-datasets', 'lrs3')

# Initialize models used in the preprocessing 
models = initialize_models(args)

# Prepare directory structure --> only this script is needed for video
dirs, splits = utils.prepare_directory_structure(dataset_dir, [args.split], video=True)

# Setup cache directories
model_combo = f"{args.text_model}"

if args.audio_model:
    model_combo += f"-{args.audio_model}"

if args.video_model:
    model_combo += f"-{args.video_model}"

# Create cache for our features and a temp directory for writing progress
dirs["cache_dir"] = os.path.join(args.output_dir, 'features', model_combo)

Initializing models...


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60 and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [113]:
print(f"\nExtracting features for {args.split} split...", flush=True)
print(f"\nCurrent shard: {args.current_shard+1}/{args.num_shards}", flush=True)
split_dirs = {k: os.path.join(v, args.split) for k, v in dirs.items()}

# Make the json directories
split_dirs["temp_dir"] = os.path.join(split_dirs["cache_dir"], 'temp')
split_dirs["errors_dir"] = os.path.join(split_dirs["cache_dir"], 'errors')


Extracting features for train split...

Current shard: 1/5


In [114]:
# Get file list 
all_fns = sorted(os.listdir(split_dirs["textgrids"]))
all_fns = [os.path.splitext(fn)[0] for fn in all_fns]

# Apply sharding logic --> divide dataset into number of shards 
if args.num_shards > 1:
    # Calculate shard size and starting/ending indices
    shard_size = math.ceil(len(all_fns) / args.num_shards)
    start_idx = args.current_shard * shard_size
    end_idx = min(start_idx + shard_size, len(all_fns))
    
    # Get only the files for the current shard
    all_fns = all_fns[start_idx:end_idx]
    
    print(f"Processing shard {args.current_shard+1}/{args.num_shards} with {len(all_fns)} files", flush=True)

Processing shard 1/5 with 23670 files


In [115]:
# Count existing json files
existing_count = 0
to_process_count = 0

process_fns = []

for fn in all_fns:
    temp_json_fn = utils.get_temp_json_path(split_dirs['temp_dir'], fn)
    # error_json_fn = utils.get_temp_json_path(split_dirs['errors_dir'], fn)

    # If the file was successfully or unsuccessfully processed, a json exists
    if (os.path.exists(temp_json_fn)): #and not args.overwrite:
        existing_count += 1
    else:
        process_fns.append(fn)
        to_process_count += 1

print(f"Found {existing_count} existing json files and {to_process_count} files to process", flush=True)

Found 0 existing json files and 23670 files to process


In [95]:
args.min_words = 3
counter = 0
max_files = 10
short_files = []

for file_name in tqdm(process_fns):
    # Load file data
    file_data = load_file_data(args, split_dirs, models, file_name)

    if file_data is None:
        continue

    if len(file_data['words']) < 4:
        short_files.append(file_name)
        counter += 1
        # continue
    if counter == max_files:
        break
    # print (len(file_data['words']))
# else:
#     message = process_single_file(args, split_dirs, models, file_name)
#     if not message:
#         continue
#     break
#         # break


  0%|          | 0/21638 [00:00<?, ?it/s]

  0%|          | 10/21638 [00:01<1:01:41,  5.84it/s]


In [116]:
file_data = load_file_data(args, split_dirs, models, process_fns[2])

# Process text
text = " ".join([word['text'] for word in file_data['words']])
text_tokens = models['tokenizer'](text)

# Get unique ids and number of tokens
word_ids, token_counts = np.unique(text_tokens.word_ids(), return_counts=True)

print (f'At time {offset}/{media_length/rate:.2f} -- over tolerance by {offset-media_length/rate:.2f}s, or {(offset * rate - media_length):.0f} idx', flush=True)


At time 2.24/2.20 -- over tolerance by 0.04s, or 1 idx


In [88]:
word_segments = utils.extract_media_segment(video_data, fps, word["start"], word["end"])


IndexError: index out of range in self

In [128]:
end_tolerance = 1
file_data = load_file_data(args, split_dirs, models, process_fns[2])

# Process text
text = " ".join([word['text'] for word in file_data['words']])
text_tokens = models['tokenizer'](text)

# Get unique ids and number of tokens
word_ids, token_counts = np.unique(text_tokens.word_ids(), return_counts=True)

words, video_data, fps = [file_data.get(item) for item in ['words', 'video_data', 'video_fps']]
processor, model = [models['video'].get(item) for item in ['processor', 'model']]

# Get word token ids
word_ids, token_counts = np.unique(text_tokens.word_ids(), return_counts=True)

segments = []

# Process each word
for i, (word, idx, n_tokens) in enumerate(zip(words, word_ids, token_counts)):
    if n_tokens > 1:
        # If current word has multiple tokens, create ratios based on length of tokens
        ratios = [len(x) for x in models['tokenizer'].batch_decode(text_tokens["input_ids"][idx:idx+n_tokens])]
        ratios = torch.tensor(ratios)
        ratios = ratios / ratios.sum()
    else:
        ratios = None

    # Extract word segments with weighted ratios
    word_segments = utils.extract_media_segment(
        video_data, 
        rate = fps, 
        onset = word["start"], 
        offset = word["end"], 
        ratios = ratios, 
        end_tolerance = end_tolerance if (i+1) == len(words) else None
    )

    segments.extend(word_segments)

NameError: name 'overby' is not defined

In [127]:
len(words)

58

In [109]:
process_video(file_data, models, text_tokens)

tensor([[-0.0210,  0.0053, -0.0112,  ..., -0.0545, -0.0241,  0.0113],
        [-0.0199,  0.0224, -0.0165,  ..., -0.0474, -0.0183,  0.0007],
        [-0.0199,  0.0300, -0.0150,  ..., -0.0447, -0.0090,  0.0012]])

In [164]:
process_single_file(args, split_dirs, models, file_name)

Mismatched number of tokens/words: 01GWGmg5jn8_50017


In [ ]:
from IPython.display import Video
Video.from_file("OUT.mp4")

In [151]:
metadata

{'video': {'fps': [25.0], 'duration': [5.52]},
 'audio': {'framerate': [16000.0], 'duration': [5.52]}}

In [154]:
file_data['words']

[{'text': 'i', 'start': 0.08, 'end': 0.46},
 {'text': 'waited', 'start': 1.14, 'end': 1.43},
 {'text': 'until', 'start': 1.43, 'end': 1.63},
 {'text': 'i', 'start': 1.63, 'end': 1.64},
 {'text': 'was', 'start': 1.64, 'end': 1.71},
 {'text': 'in', 'start': 1.71, 'end': 1.77},
 {'text': 'my', 'start': 1.77, 'end': 1.84},
 {'text': 'late', 'start': 1.84, 'end': 2.04},
 {'text': '20s', 'start': 2.04, 'end': 2.47}]

In [150]:
from torchvision.io import VideoReader
reader = VideoReader(video_path)
metadata = reader.get_metadata()

In [70]:
import librosa

In [6]:
dataset_dir = os.path.join(DATASETS_DIR, 'nlp-datasets', 'lrs3')

# Prepare directory structure --> only this script is needed for video
dirs, splits = utils.prepare_directory_structure(dataset_dir, ['train'], video=True)

video_dir = os.path.join(dataset_dir, 'video', 'train')
video_files = sorted(glob.glob(os.path.join(video_dir, "*.mp4"), recursive=True))

fn = os.path.splitext(os.path.basename(video_files[0]))[0]

KeyboardInterrupt: 

In [297]:
import os, sys
import glob
import argparse
import math
import json
import numpy as np
import torch
from tqdm import tqdm
from torchvision.io import read_video
import shutil
# from transformers import AutoImageProcessor, AutoProcessor, AutoModel, AutoTokenizer

sys.path.append('/dartfs/rc/lab/F/FinnLab/code/modeling/preproc-datasets/')
sys.path.append('/dartfs/rc/lab/F/FinnLab/code/modeling/preproc-datasets/feature-extraction')

import utils
from extract_dataset_features import (
    initialize_models,
    load_file_data,
    process_video
)

ModuleNotFoundError: No module named 'extract_dataset_features'

### Classify the language

In [214]:
from transformers import Wav2Vec2ForSequenceClassification, AutoFeatureExtractor
import torch

model_id = "facebook/mms-lid-256"

processor = AutoFeatureExtractor.from_pretrained(model_id)
model = Wav2Vec2ForSequenceClassification.from_pretrained(model_id)

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/6.61k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

In [243]:
dataset_dir = os.path.join(DATASETS_DIR, 'nlp-datasets', 'avspeech')

all_fns = [os.path.join(root, f) for root, dirs, files in os.walk(dataset_dir) for f in files if f.endswith(".mp4")]



In [241]:
import librosa
import random

In [244]:
random.shuffle(all_fns)

In [252]:
all_audio = []
for x in all_fns[:20]:
    waveform, audio_sr = utils.load_audio(x)
    waveform, audio_sr = utils.resample_audio(waveform, orig_sr=audio_sr, target_sr=16000)
    all_audio.append(waveform.squeeze())

In [254]:
# English
inputs = processor(all_audio, sampling_rate=audio_sr, padding=True, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs).logits

lang_id = torch.argmax(outputs, dim=-1)[0].item()
detected_lang = model.config.id2label[lang_id]

In [255]:
lang_ids = torch.argmax(outputs, dim=-1)

In [263]:
sorted(glob.glob(os.path.join(dataset_dir, 'clips', '*.tar')))

['/dartfs/rc/lab/F/FinnLab/datasets/nlp-datasets/avspeech/clips/xab.tar',
 '/dartfs/rc/lab/F/FinnLab/datasets/nlp-datasets/avspeech/clips/xac.tar',
 '/dartfs/rc/lab/F/FinnLab/datasets/nlp-datasets/avspeech/clips/xad.tar',
 '/dartfs/rc/lab/F/FinnLab/datasets/nlp-datasets/avspeech/clips/xae.tar',
 '/dartfs/rc/lab/F/FinnLab/datasets/nlp-datasets/avspeech/clips/xaf.tar',
 '/dartfs/rc/lab/F/FinnLab/datasets/nlp-datasets/avspeech/clips/xag.tar',
 '/dartfs/rc/lab/F/FinnLab/datasets/nlp-datasets/avspeech/clips/xah.tar',
 '/dartfs/rc/lab/F/FinnLab/datasets/nlp-datasets/avspeech/clips/xai.tar',
 '/dartfs/rc/lab/F/FinnLab/datasets/nlp-datasets/avspeech/clips/xaj.tar',
 '/dartfs/rc/lab/F/FinnLab/datasets/nlp-datasets/avspeech/clips/xak.tar',
 '/dartfs/rc/lab/F/FinnLab/datasets/nlp-datasets/avspeech/clips/xal.tar',
 '/dartfs/rc/lab/F/FinnLab/datasets/nlp-datasets/avspeech/clips/xam.tar',
 '/dartfs/rc/lab/F/FinnLab/datasets/nlp-datasets/avspeech/clips/xan.tar',
 '/dartfs/rc/lab/F/FinnLab/datasets/nl

In [258]:
for id in lang_ids:
    print (model.config.id2label[id.item()])

eng
eng
eng
eng
eng
por
eng
por
eng
eng
eng
eng
eng
eng
epo
bos
eng
eng
eng
deu


In [139]:
jsonObj['metadata'].iloc[10]

{'view_count': 12354,
 'description': '山梨簿記学院\u3000http://yboki.com/\n体験講義動画\u3000「為替手形の基本」',
 'format': {'tags': {'major_brand': 'mp42',
   'compatible_brands': 'isommp42',
   'minor_version': '0'},
  'start_time': 0.0,
  'nb_streams': 2,
  'format_name': ['3gp', '3g2', 'mov', 'mp4', 'mj2', 'm4a'],
  'bit_rate': 1116993,
  'nb_programs': 0,
  'duration': 1044.3639,
  'probe_score': 100,
  'size': 145818419},
 'video_id': 'w0Q5gH4hb7I',
 'creation_time': '2016-08-27T15:10:23.000000Z',
 'height': 720,
 'dislike_count': 2,
 'channel_id': 'UCZ-aFwJHTPY3gLQYN2S-Y3Q',
 'like_count': 30,
 'subtitles': {},
 'duration': 1044,
 'title': '為替手形の基本\u3000簿記会計',
 'tags': ['為替手形', '名宛人', '振出人', '引受人'],
 'width': 1280,
 'categories': ['Education']}